In [ ]:
import sys
base_dir = 'FILTNET-for-rapidly-predicting-fibrous-filter-media-properties' # Change this to the base directory of the project
sys.path.append(f'{base_dir}/FiltNet_training') 
import FiltNet as FN
from FiltNet import load_list
import numpy as np
from matplotlib.font_manager import FontProperties
import os

font_path = f'{base_dir}/timesbd.ttf'
font_properties = FontProperties(fname=font_path)

FileName = '_10066d'
Data = f'{base_dir}/HDF/FiltNet_1000d.h5'

ModelType=40
model = FN.loadmodel(f'{base_dir}/HDF/Trained/',ModelType)

#List=FN.prep(Data,40)
List = load_list(f'{base_dir}/list_40_10066d.h5')
TrainList, EvalList, TestList = FN.splitdata(List)

MIN,MAX=np.load(f'{base_dir}/FiltNet_training/MinMax/minmax_model40_10066d.npy')
X2_MIN,X2_MAX = np.load(f'{base_dir}/FiltNet_training/MinMax/minmax_res_model40_10066d.npy') # load voxel size minmax

In [ ]:
def feedsample(Img_dir=None, PixelSize=None,NZ=None,loadTrainingData=False,index=10):  

    if loadTrainingData==False:
        if os.path.isdir(Img_dir):
            NZ_value = np.int32(NZ)
            bgr_img = FN.BGR_img_gen(Img_dir,NZ_value=NZ_value)
    
            #normalize
            Img = bgr_img / 255.0
            res = np.log10(PixelSize * 400.0 / 256.0).astype('float32') 
            res = (res - X2_MIN) / (X2_MAX - X2_MIN)
    else:
        G=FN.gener(len(TestList),Data,List,MIN,MAX,X2_MIN,X2_MAX)
        L=next(G)
        x=L[0]
        Img = x[0][index,...] # index is the case number
        res = x[1][index,...][0]

    return Img, res

In [ ]:
#Predictions based on training database
Img,res = feedsample(loadTrainingData=True,index=97)
all_preds = FN.predict(model,Img,res,ModelType)
FN.prettyresult(all_preds,res, f'Predictions/results_{ModelType}{FileName}.txt',units='um',verbose=1)

In [ ]:
#Predictions based on semi-SEM images from 3D geometries
PixelSize = 0.5319e-6 # unit: m
all_preds = []
for i in range (1):
    Imgdir = f'{base_dir}/Imgs/MF-1/CT'
    ImgName = Imgdir.split('/')[-2]
    Img_subfolder = Imgdir.split('/')[-1]
    Imgfolder = Imgdir.split('/')[-2]
    Img,res = feedsample(Img_dir = Imgdir, PixelSize = PixelSize, NZ=227)
    preds = FN.predict(model,Img,res,ModelType)

    FN.prettyresult(preds,PixelSize, f'Predictions/{Imgfolder}/{Img_subfolder}/{ImgName}_results_{ModelType}{FileName}.txt',units='um',verbose=1)  
    all_preds.append(preds)
    
average_preds = np.mean(all_preds, axis=0)  # Compute element-wise mean across loops
average_output_file = f'Predictions/{Imgfolder}/{Img_subfolder}/Average_results_{ModelType}{FileName}.txt'
std_preds = np.std(all_preds, axis=0)
std_preds = f'Predictions/{Imgfolder}/{Img_subfolder}/Std_results_{ModelType}{FileName}.txt'
FN.prettyresult(average_preds, PixelSize, average_output_file, units='um', verbose=1)

In [ ]:
#Predictions based on real SEM images
ImgDir = f'{base_dir}/Imgs/MF-1/SEM'
outputDir = f'{ImgDir}/Patches'
Imgfolder = ImgDir.split('/')[-2]
ImgSubfolder = ImgDir.split('/')[-1]
res_surface = 0.5319e-6
res_cross = 0.5319e-6
m = 3
n = 2
windowsSize=(800,800)

patch_surface, patch_cross = FN.Patch2DImg(
    ImgDir, outputDir, res_surface, res_cross, 
    windowsSize=windowsSize,
    m=m, 
    n=n, 
    createDir=True)

res = np.log10(res_cross * windowsSize[0] / 256.0).astype('float32')
res = (res - X2_MIN) / (X2_MAX - X2_MIN)

all_preds = []
for i in range(len(patch_surface)):
    index = i % len(patch_cross)
    bgr_img = np.stack((patch_cross[index], patch_cross[index], patch_surface[i]), axis=-1)
    
    Img = bgr_img / 255.0
    preds = FN.predict(model,Img,res,ModelType)
    FN.prettyresult(preds,res_cross, f'Predictions/{Imgfolder}/{ImgSubfolder}/{windowsSize[0]}/results_{ModelType}_patch#{i}{FileName}.txt',units='um',verbose=1)
    
    all_preds.append(preds)

average_preds = np.mean(all_preds, axis=0)  # Compute element-wise mean across loops
std_preds = np.std(all_preds, axis=0)
average_output_file = f'Predictions/{Imgfolder}/{ImgSubfolder}/{windowsSize[0]}/Average_results_{ModelType}{FileName}.txt'
std_preds = f'Predictions/{Imgfolder}/{ImgSubfolder}/{windowsSize[0]}/Std_results_{ModelType}{FileName}.txt'
FN.prettyresult(average_preds, res_cross, average_output_file, units='um', verbose=1)